# Blueprint Design

In [ ]:
# truncation
truncate_item_price = True
truncate_delivery_days = False
truncate_age = False
cut_age = True

# imputation
numeric_imputer_strategy =  'median' # 'median', 'most_frequent', 'constant' 'mean'
numeric_standard_scaler_mean = True

In [ ]:
def preprocess_df(df, known_data:bool, truncate_delivery_days:bool, truncate_item_price:bool, truncate_age:bool, cut_age:bool):
    # change object variables to datatype category
    # change numeric variables from float64 to float32 (reduce memory consumption)
    # change feature return to boolean (2 categories)
    # change dates to the datetime datatype 'datetime64[ns]'
    df = transform_columns(df, known_data)
    
    # via (df['delivery_date'] - df['order_date']).dt.days
    df = add_delivery_days(df)

    if truncate_delivery_days:
        print('truncate_delivery_days')
        # via outlier_truncation(df['delivery_days'])
        # # Define upper/lower bound
        # # upper = x.quantile(0.75) + factor*IQR
        # # lower = x.quantile(0.25) - factor*IQR
        df = remove_delivery_days_outliers(df)
    
    # via df['delivery_date'].apply(lambda x: False if pd.isnull(x) else True)
    df = add_delivery_date_missing(df)

    # year<2016 is all 1994, which is suspicious
    # via df['delivery_date'].apply(lambda x: True if x.year < 2016 else False)
    df = add_delivery_date_1994_marker(df)

    # via df.loc[df['delivery_date'].dt.year < 2016,['delivery_days']] = np.nan
    df = set_delivery_date_1994_to_nan(df)
    
    # via df['brand_id'].apply(lambda x: (df['brand_id'] == x).sum())
    df = add_brand_id_count(df)

    # via df['item_id'].apply(lambda x: (df['item_id'] == x).sum())
    df = add_item_id_count(df)

    # set it all to lowercase and correct some spelling error
    # then via df['item_color'].apply(lambda x: (df['item_color'] == x).sum())
    df = add_item_color_count(df)
    
    # a practical summary for retailing size:
    # sizes_dict = {
    #     '84': 'xxs', '104': 's', '110': 's', '116': 's', '122': 'm', '128': 'm',
    #     '134': 'l', '140': 'l', '148': 'xl', '152': 'xl', '164': 'xxl', '170': 'xxl',
    #     '176': 'xxxl', '18': 'xs', '19': 's', '20': 's', '21': 'm', '22': 'm', '23': 'l',
    #     '24':  'xl', '25': 'xs', '26': 's', '27': 's', '28': 'm', '29': 'm',  '30': 'l',
    #     '31': 'l', '32': 'xl', '33': 'xxl', '34': 'xxs', '35': 'xs', '36': 'xs', '36+': 's',
    #     '37': 's', '37+': 's', '38': 's', '38+': 's', '39': 'm', '39+': 'm', '40': 'm',
    #     '40+': 'm', '41': 'm', '41+' : 'm', '42': 'l', '42+': 'l', '43': 'l', '43+': 'l',
    #     '44': 'l', '44+' : 'xl', '45' : 'xl', '45+': 'xl', '46': 'xl', '46+' : 'xl',
    #     '47' : 'xl', '48': 'xl', '49': 'xl', '50': 'xxl', '52': 'xxl', '54': 'xxl',
    #     '56': 'xxl', '58': 'xxl', 0: 'xxs', '1': 'xxs', '2': 'xxs', '2+': 'xxs', '3' : 'xxs',
    #     '3+': 'xs', '4':  'xs', '4+': 'xs', '5': 'xs', '5+':'xs', '6':'s', '6+':'s',
    #     '7':'s', '7+':'m', '8':'m', '8+':'m', '9': 'l', '9+': 'l', '10': 'l', '10+': 'xl',
    #     '11': 'xl', '11+': 'xl', '12': 'xl', '12+': 'xxl', '13': 'xxl', '14': 'xxl',
    #     36: 'xxs', 38: 'xs', 40: 's', 42: 'm', 44: 'l', 46: 'xl', 48: 'xxl',
    #     '3132': 'xxs', '3332': 'xs', '3432': 'xs', '3632': 's', '3832': 'm', '3634': 'l',
    #     '3834': 'xl', '4032': 'xl', '4034': 'xxl', '4232': 'xxxl', '80': 'xs', '85': 's',
    #     '90': 'm', '95': 'l', '100': 'xl', '105': 'xxl'
    # }
    df = convert_item_sizes(df)

    if truncate_item_price:
        print('truncate_price_size')
        # via outlier_truncation(df['item_price'])
        df = truncate_item_price_outliers(df)

    # via df['user_dob'].apply(calculate_age)
    # today = date.today()
    # return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    df = add_age(df)

    if truncate_age:
        print('truncate_age')
        # via outlier_truncation(df['age'])
        df = truncate_age_outliers(df)
    if cut_age:
        print('cut_age')
        # via df.loc[df['age'] > 95,'age'] = np.nan
        df = cut_age_outliers(df)
    
    # via df['user_dob'].apply(lambda x: False if pd.isnull(x) else True)
    df = add_dob_missing(df)

    # via df['been_member_for'] = (df['order_date']-df['user_reg_date'] ).dt.days
    df = add_been_member_for(df)

    # labels = ['fresh_member', 'new_member', 'member', 'old_member']
    # cut_bins = [-5, 150, 300, 450, 1000]
    # via df['member'] = pd.cut(df['been_member_for'], bins=cut_bins, labels=labels)
    df = add_member_category(df)

    print(df.info())
    return df   

def transform_columns(df_known, known_data:bool):
    # change object variables to datatype category
    df_known['item_size'] = df_known['item_size'].astype('category')
    df_known['item_color'] = df_known['item_color'].astype('category')
    df_known['user_title'] = df_known['user_title'].astype('category')
    df_known['user_state'] = df_known['user_state'].astype('category')
    # change all numeric variables from float64 to float32 to reduce memory consumption
    df_known['item_price'] = df_known['item_price'].astype(np.float32)
    df_known['item_price'] = df_known['item_price'].apply(lambda x:("%.2f" % round(x, 2)))
    df_known['item_price'] = df_known['item_price'].astype(np.float32)
    df_known['brand_id'] = df_known['brand_id'].astype(np.int32)
    df_known['user_id'] = df_known['user_id'].astype(np.int32)
    df_known['item_id'] = df_known['item_id'].astype(np.int32)
    if known_data:
        # since the feature return has only two values, we convert it to boolean
        df_known['return'] = df_known['return'].astype('bool')
    # transform all dates to the datetime datatype
    df_known['order_date'] = df_known['order_date'].astype('datetime64[ns]')
    df_known['delivery_date'] = df_known['delivery_date'].astype('datetime64[ns]')
    df_known['user_dob'] = df_known['user_dob'].astype('datetime64[ns]')
    df_known['user_reg_date'] = df_known['user_reg_date'].astype('datetime64[ns]')
    return df_known

df = transform_columns(df, known_data=True)
df.info()

# <class 'pandas.core.frame.DataFrame'>
# Int64Index: 100000 entries, 1 to 100000
# Data columns (total 13 columns):
#  #   Column         Non-Null Count   Dtype         
# ---  ------         --------------   -----         
#  0   order_date     100000 non-null  datetime64[ns]
#  1   delivery_date  90682 non-null   datetime64[ns]
#  2   item_id        100000 non-null  int32         
#  3   item_size      100000 non-null  category      
#  4   item_color     100000 non-null  category      
#  5   brand_id       100000 non-null  int32         
#  6   item_price     100000 non-null  float32       
#  7   user_id        100000 non-null  int32         
#  8   user_title     100000 non-null  category      
#  9   user_dob       91275 non-null   datetime64[ns]
#  10  user_state     100000 non-null  category      
#  11  user_reg_date  100000 non-null  datetime64[ns]
#  12  return         100000 non-null  bool          
# dtypes: bool(1), category(4), datetime64[ns](4), float32(1), int32(3)
# memory usage: 5.8 MB